In [ ]:
import pandas as pd
import os

In [ ]:
work_dir = '/data/amandeep/nih-dataset'
person_dir = f'{work_dir}/person'
org_dir = f'{work_dir}/organization'

combined_dir = f'{work_dir}/combined'

os.environ['STORE'] = f'{combined_dir}/wikidata.sqlite3.db'
os.environ['kypher'] = "kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['COMBINED'] = combined_dir

In [ ]:
!mkdir -p $combined_dir

In [ ]:
!kgtk cat -i $person_dir/replace_nodes_mapping.tsv -i $org_dir/replace_nodes_mapping.tsv \
-o $combined_dir/replace_nodes_mapping.tsv

In [ ]:
!kgtk cat -i $person_dir/nih-person-kgtk.tsv -i $org_dir/nih-org-kgtk.tsv \
-o $combined_dir/nih-kgtk.tsv

In [ ]:
!kgtk replace-nodes -i $combined_dir/nih-kgtk.tsv -o $combined_dir/nih-kgtk-wikidata-qnodes.tsv --mapping-file $combined_dir/replace_nodes_mapping.tsv

In [ ]:
!kgtk add-id --id-style wikidata -i $combined_dir/nih-kgtk-wikidata-qnodes.tsv -o $combined_dir/nih-kgtk-wikidata-qnodes-ids.tsv

In [ ]:
!kgtk sort -i $combined_dir/nih-kgtk-wikidata-qnodes-ids.tsv -o $combined_dir/nih-kgtk-wikidata-qnodes-ids-sorted.tsv

In [ ]:
!$kypher -i "$COMBINED/nih-kgtk-wikidata-qnodes-ids-sorted.tsv" --as nih \
--limit 10

In [ ]:
%%time
!$kypher -i nih \
--match '\
    nih: (person1)-[:P108]->(org), \
    nih: (person2)-[:P108]->(org),  \
    nih: (person1)-[:label]->(person1_label), \
    nih: (person2)-[:label]->(person2_label), \
    nih: (org)-[:label]->(org_label)' \
--where 'person1 < person2' \
--return 'distinct person1 as node1, person1_label as `node1;label`, "Pcoworker" as label, person2 as node2, person2_label as `node2;label`, org as organization, org_label as `organization;label`' \
-o "$COMBINED"/coworkers.nih.tsv.gz